In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import os

def fetch_and_process_data(ticker, start_date, end_date):
    """
    Fetch historical data for a given ticker, calculate volatility, and return a DataFrame.
    
    Parameters:
    - ticker (str): Stock or ETF ticker symbol (e.g., 'TSLA', 'BND', 'SPY').
    - start_date (str): Start date in 'YYYY-MM-DD' format.
    - end_date (str): End date in 'YYYY-MM-DD' format.
    
    Returns:
    - DataFrame with Date, Open, High, Low, Close, Adj Close, Volume, and Volatility.
    """
    try:
        # Download historical data using yfinance
        df = yf.download(ticker, start=start_date, end=end_date, interval="1d", auto_adjust=False, progress=False)
        
        if df.empty:
            raise ValueError(f"No data retrieved for {ticker}. Check ticker symbol or date range.")
        
        # Reset index to make Date a column
        df.reset_index(inplace=True)
        
        # Select required columns
        df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
        
        # Calculate daily returns (based on Adj Close)
        df['Daily_Return'] = df['Adj Close'].pct_change()
        
        # Calculate daily volatility (standard deviation of daily returns over a 21-day rolling window)
        df['Volatility'] = df['Daily_Return'].rolling(window=21).std() * np.sqrt(252)  # Annualized volatility
        
        # Drop rows with NaN values (e.g., first row after pct_change or rolling window)
        df = df.dropna()
        
        # Round numerical columns to 2 decimal places for readability
        df[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Daily_Return', 'Volatility']] = \
            df[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Daily_Return', 'Volatility']].round(2)
        
        return df
    
    except Exception as e:
        print(f"Error fetching data for {ticker}: {str(e)}")
        return None

def main():
    # Define tickers and date range
    tickers = ['TSLA', 'BND', 'SPY']
    start_date = '2015-07-01'
    end_date = '2025-07-31'
    
    # Create a directory to save the data
    output_dir = 'financial_data'
    os.makedirs(output_dir, exist_ok=True)
    
    # Fetch and process data for each ticker
    for ticker in tickers:
        print(f"Fetching data for {ticker}...")
        df = fetch_and_process_data(ticker, start_date, end_date)
        
        if df is not None:
            # Save to CSV
            output_file = os.path.join(output_dir, f"{ticker}_historical_data_2015_2025.csv")
            df.to_csv(output_file, index=False)
            print(f"Data for {ticker} saved to {output_file}")
            
            # Display first few rows for verification
            print(f"\nFirst 5 rows of {ticker} data:")
            print(df.head())
            print(f"Shape of {ticker} data: {df.shape}\n")
        else:
            print(f"Failed to fetch data for {ticker}")

if __name__ == "__main__":
    main()

Fetching data for TSLA...
Data for TSLA saved to financial_data\TSLA_historical_data_2015_2025.csv

First 5 rows of TSLA data:
Price        Date   Open   High    Low  Close Adj Close     Volume  \
Ticker              TSLA   TSLA   TSLA   TSLA      TSLA       TSLA   
21     2015-07-31  17.84  17.96  17.67  17.74     17.74   33339000   
22     2015-08-03  17.75  17.78  17.14  17.33     17.33   38302500   
23     2015-08-04  17.33  17.78  17.22  17.75     17.75   35287500   
24     2015-08-05  17.57  18.07  17.36  18.01     18.01   93214500   
25     2015-08-06  16.64  17.00  15.74  16.41     16.41  219357000   

Price  Daily_Return Volatility  
Ticker                          
21            -0.00       0.45  
22            -0.02       0.43  
23             0.02       0.44  
24             0.01       0.41  
25            -0.09       0.49  
Shape of TSLA data: (2514, 9)

Fetching data for BND...
Data for BND saved to financial_data\BND_historical_data_2015_2025.csv

First 5 rows of BND dat